# ASTR 19 – Final Project    
**University of California, Santa Cruz**  
**Instructor: Prof. Brant Richardson**  
**Student: Nathaniel Frazer**    
**ID: 1792903** 

---

## Project Overview

- Detect sources in astronomical FITS images  
- Measure statistical properties of the detected objects  
- Assess the significance of bright outliers relative to a model  
- Construct scientific visualizations, including a 3-color false image of the HUDF  
- Compare results across different datasets and wavelengths  

---

In [ ]:
# Import NumPy for numerical operations and array handling
import numpy as np
# Import SEP (Source Extraction in Python) for background estimation and source extraction
import sep

In [ ]:
# Astropy provides FITS file support (the standard format for astronomy images)
import astropy
from astropy.io import fits

# Matplotlib for plotting and visualization
import matplotlib.pyplot as plt
from matplotlib import rcParams

# Make all plots appear inline in the notebook
%matplotlib inline

# Set a default figure size for all plots to make them easier to view
rcParams['figure.figsize'] = [10., 8.]

---
## Step 5 – Loading the HUDF f105w Image
Using `astropy.io.fits`, I load the f105w image for background estimation, source detection, and photometric analysis.

In [ ]:
# Read the HUDF f105w FITS image.
data = fits.getdata('hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits')

# FITS new byte order required
new_data = data.byteswap(inplace=True).newbyteorder()
# 'new_data' will be used for background estimation and source extraction.
new_data = data.byteswap().newbyteorder()

In [ ]:
# Compute the mean and standard deviation of the image pixel values.
# This is used to set a reasonable display range.
m, s = np.mean(new_data), np.std(new_data)

# Display the HUDF f105w image with a grayscale colormap.
# vmin and vmax define a (mean ± 1σ) contrast stretch.
plt.imshow(
    new_data,
    interpolation='nearest',
    cmap='gray',
    vmin=m - s,
    vmax=m + s,
    origin='lower'
)

# Add a colorbar to show the mapping from pixel values to grayscale intensity.
plt.colorbar()

# Save the figure for documentation.
plt.savefig('band_image_hubble_original.png')

# Show the plot in the notebook.
plt.show()

---
## Step 6 – Applying SEP to the HUDF f105w Image

1. Estimate and subtract a spatially varying background using `sep.Background`.
2. Compute a background RMS map to characterize noise levels.
3. Subtract the background from the original image to produce `data_sub`.
4. Run `sep.extract` on the background–subtracted image to detect sources.
5. Count the total number of detected sources.
6. Extract the **flux** values (`objects["flux"]`) for all sources and make
   a histogram of their fluxes.

In [ ]:
# Use SEP to estimate a smoothly varying background across the f105w image.
bkg = sep.Background(new_data)

In [ ]:
# Print the global (mean) background value estimated by SEP.
print(bkg.globalback)

# Print the global RMS (noise) of the background.
print(bkg.globalrms)

In [ ]:
# Evaluate the SEP background model
bkg_image = bkg.back()

In [ ]:
# Display the SEP background model using a grayscale color scale.
plt.imshow(
    bkg_image,
    interpolation='nearest',
    cmap='gray',
    origin='lower'
)

# Add a colorbar to visualize intensity values.
plt.colorbar()

# Save the figure for documentation
plt.savefig('hubble_background.png')

# Display the figure in the notebook.
plt.show()

In [ ]:
# Evaluate the background RMS (noise) across the image.
bkg_rms = bkg.rms()

In [ ]:
# Display the background RMS (noise) map using a grayscale colormap.
plt.imshow(
    bkg_rms,
    interpolation='nearest',
    cmap='gray',
    origin='lower'
)

# Add a colorbar to show the noise scale.
plt.colorbar()

# Save the figure documentation.
plt.savefig('hubble_background_noise.png')

# Display the figure in the notebook.
plt.show()

In [ ]:
# Subtract the background model from the original image.

data_sub = new_data - bkg_image

In [ ]:

objects = sep.extract(
    data_sub,            # Detect sources in the background-subtracted image using SEP.
    thresh=3*bkg_rms,    # Threshold = 3 * RMS noise, meaning a pixel must be 3σ above the background to be considered part of a source
    minarea=7,           # minarea = 7 ensures that very small noise fluctuations are ignored.
    err=bkg.globalrms    # err = bkg.globalrms provides the noise level for proper thresholding.
)

# 'objects' is now a catalog of detected astronomical sources.

In [ ]:
# Print the number of detected sources in the HUDF image.
print(len(objects))
# Each entry in 'objects' corresponds to one identified source.

In [ ]:
from matplotlib.patches import Ellipse

# Create a figure and axis for plotting
fig, ax = plt.subplots()

# Compute mean and standard deviation for contrast stretching
m, s = np.mean(data_sub), np.std(data_sub)

# Display the background-subtracted HUDF image
im = ax.imshow(
    data_sub,
    interpolation='nearest',
    cmap='gray',
    vmin=m - s,
    vmax=m + s,
    origin='lower'
)

# Overlay an ellipse for each detected object
for i in range(len(objects)):
    e = Ellipse(
        xy=(objects['x'][i], objects['y'][i]),        # source centroid
        width=6 * objects['a'][i],                    # scaled semi-major axis
        height=6 * objects['b'][i],                   # scaled semi-minor axis
        angle=objects['theta'][i] * 180.0 / np.pi     # convert radians → degrees
    )
    e.set_facecolor('none')  # no fill
    e.set_edgecolor('red')   # outline color
    ax.add_artist(e)

# Save the visualization for documentation
plt.savefig('hubble_background_subtracted_image.png')

# Show the plot
plt.show()

In [ ]:
# Extract the flux values from the SEP object catalog.
# These fluxes represent total brightness measured within adaptive elliptical apertures.
fluxes = objects["flux"]

# Create the histogram figure
plt.figure(figsize=(8, 5))

# Plot a histogram with many bins and logarithmic y-scale to capture the dynamic range.
plt.hist(
    fluxes,
    bins=1000,          # fine binning to reveal structure
    edgecolor="red",    # red edges for visual clarity
    density=False,
    log=True            # log-scale to show faint source counts
)

# Labeling and axis limits
plt.xlabel("Flux")
plt.xlim(0, 1300)       # focus on main flux range
plt.ylim(0, 500)        # adjusted based on detection density
plt.ylabel("Number of Sources")
plt.title("Histogram of Fluxes (f105w)")

# Save the plot for the final project report
plt.savefig("flux_histogram.png", dpi=150)

# Display the figure
plt.show()

---
## Step 7 – Flux Statistics and Brightest Outlier

1. Compute the **mean**, **median**, and **standard deviation** of the flux values.
2. Identify the **brightest source** (largest flux in the catalog).
3. Determine the pixel coordinates of the brightest source in the image.
4. Calculate how many **standard deviations** above the mean this brightest source lies.
5. Graph the location of this outlier on the background–subtracted HUDF image.

In [ ]:
# Compute basic statistics of the flux distribution
mean = np.mean(fluxes)
median = np.median(fluxes)
std = np.std(fluxes)

print(f"Mean: {mean}")
print(f"Median: {median}")
print(f"Standard Deviation: {std}")

# Identify the brightest object (largest flux in the array)


# Maximum flux value among all detected objects
max_flux = np.max(fluxes)

# Index of this brightest object in the catalog
max_index = np.argmax(fluxes)

# Coordinates of the brightest object
coords = (objects['x'][max_index], objects['y'][max_index])

# Standard deviations above the mean the maximum fux is
sigma_away = (max_flux - mean) / std

print(f"Brightest Source Flux: {max_flux}")
print(f"Coordinates of Brightest Source (x, y): {coords}")
print(f"Standard Deviations Above the Mean: {sigma_away}")

In [ ]:
# Extract the x and y coordinates of the brightest object using its index.
x_out = objects["x"][max_index]
y_out = objects["y"][max_index]

# Print a summary of the brightest source information
print("Brightest flux:", max_flux)
print("Coordinates (x, y):", x_out, y_out)

In [ ]:
# Create a new figure for visualizing the brightest source on the image
plt.figure(figsize=(8, 8))

# Display the background-subtracted HUDF f105w image
# Using percentile-based scaling improves contrast and visibility
plt.imshow(
    data_sub,
    origin="lower",
    cmap="gray",
    vmin=np.percentile(data_sub, 5),
    vmax=np.percentile(data_sub, 99)
)

# Add a colorbar to show the relative flux scale
plt.colorbar(label="Flux")

# Plot a red circle marking the brightest detected source
plt.plot(
    x_out, y_out,
    marker="o",
    color="red",
    markersize=10,
    label="Brightest Source"
)

# Axis labels and title
plt.xlabel("X Pixel")
plt.ylabel("Y Pixel")
plt.title("Brightest Outlier Source in HUDF f105w")

# Include legend for clarity
plt.legend()

# Tight layout for clean saving
plt.tight_layout()

# Save the figure to file for your final project submission
plt.savefig("brightest_source_marker.png", dpi=150)

# Display the figure
plt.show()

---
## Step 8 – 3–Color False Image of the HUDF

- **Red channel**  - f160w  
- **Green channel** - f125w  
- **Blue channel**  - f105w  

1. Load the f105w, f125w, and f160w HUDF images from the same archive.
2. Apply a logarithmic rescaling function to each image to compress dynamic range and highlight both faint and bright features.
3. Clip each rescaled image to a suitable [min, max] display range.
4. Normalize each band to the interval (0, 1).
5. Stack the three normalized arrays into an RGB image.
6. Display and save the composite as a PNG.

In [ ]:
# Load HUDF images for the RGB composite.

# Blue channel
f105w = fits.getdata("hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits")

# Green channel
f125w = fits.getdata("hlsp_hudf12_hst_wfc3ir_udfmain_f125w_v1.0_drz.fits")

# Red channel
f160w = fits.getdata("hlsp_hudf12_hst_wfc3ir_udfmain_f160w_v1.0_drz.fits")

In [ ]:
def rescale_image(data):
    
    # Make a copy of the input image so the original is not modified
    pdata_tmp = data.copy()

    # Compute the mean pixel value
    m = np.nanmean(pdata_tmp)

    # Define a lower cutoff and range based on the mean brightness
    vplmin = m / 2.0             # Minimum allowed pixel value
    vpmin = np.log10(vplmin)     # Lower bound in log space
    vpmax = np.log10(m * 100.0)  # Upper bound in log space

    # Clip values below the cutoff
    pdata_tmp[pdata_tmp < vplmin] = vplmin

    # Compress the dynamic range
    pdata_tmp = np.log10(pdata_tmp)

    # rescaled image and limit
    return pdata_tmp, vpmin, vpmax

In [ ]:
# Apply the logarithmic function to each HUDF filter.

# Red channel (f160w)
r_res, rmin, rmax = rescale_image(f160w)

# Green channel (f125w)
g_res, gmin, gmax = rescale_image(f125w)

# Blue channel (f105w)
b_res, bmin, bmax = rescale_image(f105w)

In [ ]:
# Clip display range.
# Necessary before normalization

# Red channel (f160w)
r_res[r_res < rmin] = rmin
r_res[r_res > rmax] = rmax

# Green channel (f125w)
g_res[g_res < gmin] = gmin
g_res[g_res > gmax] = gmax

# Blue channel (f105w)
b_res[b_res < bmin] = bmin
b_res[b_res > bmax] = bmax

In [ ]:

# Initialize an empty RGB image cube
rgb = np.zeros((r_res.shape[0], r_res.shape[1], 3))

# Normalize each band to the range [0, 1] and assign channels

# Red channel  (f160w)
rgb[:, :, 0] = (r_res - rmin) / (rmax - rmin)

# Green channel (f125w)
rgb[:, :, 1] = (g_res - gmin) / (gmax - gmin)

# Blue channel  (f105w)
rgb[:, :, 2] = (b_res - bmin) / (bmax - bmin)


# Display and save the RGB composite

plt.figure(figsize=(8, 8))
plt.axis("off")  # Hide axes for a cleaner visualization
plt.title("HUDF 3-Color False Image (f160w, f125w, f105w)")

plt.imshow(rgb)
plt.savefig("HUDF_rbg_color.png", bbox_inches="tight", pad_inches=0, dpi=300)
plt.show()

---